<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Classification and KNN with NHL data

_Authors: Joseph Nelson (DC)_

---

Below you will practice KNN classification on a dataset of NHL statistics.

You will be predicting the `Rank` of a team from predictor variables of your choice.

In [37]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [38]:
# web location:
local_csv = '../data/NHL_Data_GA.csv'


### 1. Load the NHL data

In [39]:
# A:
pd.reset_option('all')
nhl = pd.read_csv(local_csv ,index_col=0)
nhl.describe()



: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.



/usr/local/Cellar/jupyterlab/1.2.4/libexec/lib/python3.7/site-packages/pandas/_config/config.py:620: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  warnings.warn(d.msg, FutureWarning)


,PTS,Rank,GF,GA,GF60,GA60,GF%,SF,SA,SF60,...,FF%,CF,CA,CF60,CA60,CF%,Sh%,Sv%,PDO,PIM
count,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,...,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000,90.000000
mean,91.977778,2.022222,83.288889,83.288889,2.442222,2.444000,49.981111,1068.333333,1068.333333,31.252222,...,49.966667,1973.466667,1973.466667,57.735556,57.798889,49.972222,7.814556,92.182556,999.988889,990.966667
std,12.524114,0.820767,10.376339,9.694484,0.325331,0.313522,4.644554,95.929047,75.514118,2.237637,...,2.797913,176.468299,154.148928,4.124476,4.291106,2.844313,0.866942,0.928621,12.292772,178.049321
min,62.000000,1.000000,57.000000,64.000000,1.700000,1.730000,38.000000,815.000000,868.000000,25.800000,...,43.100000,1565.000000,1572.000000,49.500000,46.700000,43.700000,5.900000,89.830000,978.000000,689.000000
25%,82.250000,1.000000,76.000000,75.500000,2.232500,2.202500,46.825000,1011.500000,1022.250000,29.550000,...,47.775000,1855.250000,1877.000000,54.275000,54.600000,47.925000,7.235000,91.555000,992.000000,881.250000
50%,92.500000,2.000000,84.000000,84.000000,2.400000,2.495000,49.700000,1072.000000,1072.000000,31.400000,...,50.050000,1981.500000,1961.000000,58.050000,58.350000,50.400000,7.730000,92.250000,1000.500000,960.000000
75%,102.000000,3.000000,90.000000,89.000000,2.600000,2.670000,53.625000,1143.000000,1125.750000,32.775000,...,51.775000,2112.750000,2077.250000,60.850000,60.400000,52.000000,8.270000,92.870000,1007.750000,1101.500000
max,121.000000,3.000000,115.000000,107.000000,3.450000,3.240000,61.200000,1311.000000,1245.000000,35.600000,...,58.100000,2341.000000,2332.000000,64.900000,67.500000,57.100000,10.340000,93.940000,1034.000000,1515.000000


In [86]:
nhl.head()

,PTS,Rank,TOI,GF,GA,GF60,GA60,GF%,SF,SA,...,FF%,CF,CA,CF60,CA60,CF%,Sh%,Sv%,PDO,PIM
Team,,,,,,,,,,,,,,,,,,,,,
Washington10,121,1,2001:52:00,115,73,3.45,2.19,61.2,1112,1047,...,51.3,2138,1935,64.1,58.0,52.5,10.34,93.03,1034,1269
Vancouver11,117,1,2056:14:00,94,72,2.74,2.10,56.6,1143,1053,...,53.1,2144,1870,62.6,54.6,53.4,8.22,93.16,1014,985
San Jose10,113,1,1929:54:00,90,68,2.80,2.11,57.0,1065,1039,...,50.9,1985,1876,61.7,58.3,51.4,8.45,93.46,1019,1195
Chicago10,112,1,2020:23:00,104,83,3.09,2.46,55.6,1186,868,...,58.1,2093,1572,62.2,46.7,57.1,8.77,90.44,992,966
Vancouver12,111,1,2052:02:00,86,74,2.51,2.16,53.8,1078,1115,...,51.0,2085,1880,61.0,55.0,52.6,7.98,93.36,1013,1049


### 2. Perform any required data cleaning. Do some EDA.

In [40]:
# A:
nhl.isnull().sum()


PTS     0
Rank    0
TOI     0
GF      0
GA      0
GF60    0
GA60    0
GF%     0
SF      0
SA      0
SF60    0
SA60    0
SF%     0
FF      0
FA      0
FF60    0
FA60    0
FF%     0
CF      0
CA      0
CF60    0
CA60    0
CF%     0
Sh%     0
Sv%     0
PDO     0
PIM     0
dtype: int64

### 3. Set up the `Rank` variable as your target. How many classes are there?

In [41]:
# A:
nhl['Rank'].nunique()

3

In [42]:
nhl['Rank'].unique()

array([1, 2, 3])

In [44]:
y = nhl['Rank']
y

Team
Washington10    1
Vancouver11     1
San Jose10      1
Chicago10       1
Vancouver12     1
               ..
Florida11       3
Columbus11      3
Colorado12      3
Edmonton10      3
Edmonton11      3
Name: Rank, Length: 90, dtype: int64

### 4. What is the baseline accuracy?

In [88]:
# A:
# y.value_counts()/y.sum()
nhl.Rank.mean()

2.022222222222222

### 5. Choose 4 features to be your predictor variables and set up your design matrix.

In [47]:
# A:
nhl.columns

Index(['PTS', 'Rank', 'TOI', 'GF', 'GA', 'GF60', 'GA60', 'GF%', 'SF', 'SA',
       'SF60', 'SA60', 'SF%', 'FF', 'FA', 'FF60', 'FA60', 'FF%', 'CF', 'CA',
       'CF60', 'CA60', 'CF%', 'Sh%', 'Sv%', 'PDO', 'PIM'],
      dtype='object')

In [48]:
feature_cols = ['SA60', 'SF%', 'FF', 'FA'];
X = nhl[feature_cols]
X

,SA60,SF%,FF,FA
Team,,,,
Washington10,31.4,51.5,1526,1449
Vancouver11,30.7,52.0,1602,1414
San Jose10,32.3,50.6,1493,1442
Chicago10,25.8,57.7,1599,1151
Vancouver12,32.6,49.2,1504,1442
...,...,...,...,...
Florida11,33.4,49.0,1574,1616
Columbus11,31.1,50.5,1480,1416
Colorado12,32.4,50.1,1557,1583


### 6. Fit a `KNeighborsClassifier` with 1 neighbor using the target and predictors.

In [57]:
# A:
from sklearn.neighbors import KNeighborsClassifier
knn =  KNeighborsClassifier(n_neighbors = 1)
knn.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [58]:
pred = knn.predict(X)
pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3])

### 7. Evaluate the accuracy of your model.
- Is it better than baseline?
- Is it legitimate?

In [70]:
# A:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y, pred))
print(classification_report(y, pred))

[[29  0  0]
 [ 0 30  0]
 [ 0  0 31]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        29
           2       1.00      1.00      1.00        30
           3       1.00      1.00      1.00        31

    accuracy                           1.00        90
   macro avg       1.00      1.00      1.00        90
weighted avg       1.00      1.00      1.00        90



### 8. Create a 50-50 train-test-split of your target and predictors. Refit the KNN and assess the accuracy.

In [73]:
# A:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=100, test_size=0.5)

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

print(accuracy_score(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))




0.4222222222222222
[[10  4  1]
 [ 7  5  3]
 [ 7  4  4]]


### 9. Evaluate the test accuracy of a KNN where K == number of rows in the training data.

In [84]:
# A:
kNum = X_train.shape[0]
knn = KNeighborsClassifier(n_neighbors=kNum)
knn.fit(X_train, y_train)
y_pred2 = knn.predict(X_test)
print(accuracy_score(y_test, y_pred2))

0.3333333333333333


### 10. Fit the KNN at values of K from 1 to the number of rows in the training data.
- Store the test accuracy in a list.
- Plot the test accuracy vs. the number of neighbors.

In [12]:
# A:

### 11. Fit KNN across different values of K and plot the mean cross-validated accuracy with 5 folds.

In [13]:
# A:

### 12. Standardize the predictor matrix and cross-validate across the different K.
- Plot the standardized mean cross-validated accuracy against the unstandardized. Which is better?
- Why?

In [14]:
# A: